In [1]:
import time
import pandas as pd


#实际中遇到的日期类型 经常有‘2018-09-08-17:45:23’这种类型 ，如何进行加减乘除运算呢？

In [5]:
?pd.to_datetime

#我们先将csv，dataframe 的object的数据类型转化成datetime64，再使用pd.to_datetime进行操作

#for eample：

In [6]:
df = pd.DataFrame({'year': [2015, 2016],
                       'month': [2, 3],
                       'day': [4, 5],
                        'hour':[2,3],
                         'minute':[4,6],
                         'second':[5,6]})

df = pd.to_datetime(df)
df

0   2015-02-04 02:04:05
1   2016-03-05 03:06:06
dtype: datetime64[ns]

In [7]:
df = df.apply(lambda x:time.mktime(x.timetuple()))  #将时间类型转化成时间戳形式
df

0    1.422987e+09
1    1.457118e+09
dtype: float64

In [8]:
cc = pd.to_datetime(df.loc[1]) - pd.to_datetime(df.loc[0])  #进行相减操作

#df = pd.to_datetime(df)  #将时间戳转化为日期格式
df = df.apply(lambda x:time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(x)))   #这一种也可以 2015-02-04 02:04:05
df['3'] = cc 
df

0    2015-02-04 02:04:05
1    2016-03-05 03:06:06
3               34131721
dtype: object

In [9]:
#最后如果想把df['3']也变成day：hour：minute：second 我们可以直接这样操作
df = pd.DataFrame({'year': [2015, 2016],
                       'month': [2, 3],
                       'day': [4, 5],
                        'hour':[2,3],
                         'minute':[4,6],
                         'second':[5,6]})

df = pd.to_datetime(df)
df

0   2015-02-04 02:04:05
1   2016-03-05 03:06:06
dtype: datetime64[ns]

In [11]:
df['3'] = pd.to_datetime(df.loc[1],format ='%Y-%m-%d %H:%M:%S',unit = 's' ) - pd.to_datetime(df.loc[0],format ='%Y-%m-%d %H:%M:%S',unit = 's')  #进行相减操作
df

0    2015-02-04 02:04:05
1    2016-03-05 03:06:06
3      395 days 01:02:01
dtype: object

# 具体案例分析

In [2]:
df = pd.read_csv('201601.csv')
df[:5]

,入口网络编号,入口站编号,入口日期及时间,出口网络编号,出口站编号,出口日期及时间,出口车道编号,车型代码,车种代码,里程,...,车货总重,限重,超限率,是否绿色通道车辆代码,免费类型代码,路径标识,是否ETC车道代码,ETC车辆电子标签OBU编号,支付方式代码,车牌号
0,5201,10753,2016-01-11 16:18:18,5201,9217,2016-01-11 17:37:02,82,1,0,118.189,...,1450,14000,-89.64,0,0,NaN,0,0,0,贵H75495
1,5201,8973,2016-01-11 16:50:09,5201,8970,2016-01-11 17:30:23,84,1,0,64.143,...,1450,14000,-89.64,0,0,NaN,0,0,0,贵J31588
2,5201,5123,2016-01-11 15:23:22,5201,8970,2016-01-11 17:31:30,84,14,1,113.067,...,37900,32000,18.44,0,0,NaN,0,0,0,贵J92622
3,5201,8968,2016-01-11 16:58:25,5201,8970,2016-01-11 17:31:14,83,1,0,54.720,...,1750,14000,-87.50,0,0,NaN,0,0,0,贵J12865
4,5201,8968,2016-01-11 16:55:18,5201,8970,2016-01-11 17:31:46,83,1,0,54.720,...,1400,14000,-90.00,0,0,NaN,0,0,0,贵J48408


我们要做一个出口减入入口的时间，并且插入新的列，这时候需要用apply广播一下，然后对每一列进行转datetime操作，最后进行相减，代码如下：

In [3]:
df['差值'] = df.apply(lambda row: pd.to_datetime(row['出口日期及时间'])- pd.to_datetime(row['入口日期及时间']),axis =1)
df['差值'][:5]

0   01:18:44
1   00:40:14
2   02:08:08
3   00:32:49
4   00:36:28
Name: 差值, dtype: timedelta64[ns]